In [ ]:
# Import used packages
%matplotlib notebook
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from matplotlib import animation
import networkx as nx
import random
import copy
import pandas as pd
from scipy.integrate import solve_ivp
from scipy import optimize

# Implementation of SEIR model
## Model
The SEIR model describes the propagation of a disease in 4 stages: susceptible, exposed, infected and recovered. Each state has different properties. At the start most people are susceptible, these can get exposed when in contact with someone who is infected. Exposed people cannot infect other people yet. They will get infectious after some time, dependent on the incubation period. Infected people can recover, when recovered they are immune to the dissease.
## Parameters
We used $\beta$ as the probability to become exposed when in contact with 1 infected person, $\delta$ is the probability of becoming infected when exposed and $\gamma$ is the probability to recover when infected.
## Datastructure
To contain all the data, we used a dictionairy 'Network'. This contains the following keys: <br>
- 'nodes': int, the amount of nodes <br>
- 'pos': list, list of lists of length 2 containing the x and y positions of the nodes <br>
- 'links': list, list of lists containing the numbers corresponding to the nodes connected to the i'th node <br>
- 'currentState': list, list of the stage of the i'th node at the current timestep <br>
- 'currentColor': list, list of the color of the i'th node at the current timestep <br>
- 'newState': list, list of the stage of the i'th node at the next timestep <br>
- 'dataStates': list (of lists), list of all the lists passing 'currentState' <br>
- 'dataColors': list (of lists), list of all the lists passing 'currentColor' <br>
- 'time': list, list of consecutive integers representing the timesteps <br>
- 'S': list, list of how many nodes are in the susceptible stage at the i'th timestep <br>
- 'E': list, list of how many nodes are in the exposed stage at the i'th timestep <br>
- 'I': list, list of how many nodes are in the infected stage at the i'th timestep <br>
- 'R': list, list of how many nodes are in the recovered stage at the i'th timestep <br>
- 'socialDistanceTimestep': int, the timestep at which social distancing occurs (defaults to np.nan)

In [ ]:
# Declare global parameters
class ST_base():
    pass

ST = ST_base()
ST.S = 0
ST.E = 1
ST.I = 2
ST.R = 3

class ST_color():
    pass

STcolor = ST_color()
STcolor.S = 'palegreen'
STcolor.E = 'orchid'
STcolor.I = 'tomato'
STcolor.R = 'deepskyblue'

class PL_attributes():
    pass

PL = PL_attributes()
PL.edgeColor = 'grey'
PL.edgeAlpha = 0.4
PL.nodeSize = 80
PL.zoom = 5

In [ ]:
def createWattsStrogatz(nodes, k, p):
    k = roundtoeven(k)
    linksTemp = []
    for i in range(nodes):
        linksI = []
        for j in range(i - int(k/2), i):
            linksI.append(j%nodes)
        for j in range(i + 1, i + int(k/2) + 1):
            linksI.append(j%nodes)
        linksTemp.append(linksI)
    links = [ [] for i in range(nodes) ]
    for i in range(nodes - 1):
        for j in linksTemp[i]:
            if j>i:
                randN = np.random.uniform()
                if randN <= p/2:
                    randNode = np.random.randint(0, nodes)
                    while randNode in linksTemp[i] or randNode == i:
                        randNode = np.random.randint(0, nodes)
                    #randNode = random.choice(list(range(0, nodes)).remove(links[i], i)) <- sneller als dit lukt
                    links = addlink(links, i, randNode)
                elif randN <= p:
                    randNode = np.random.randint(0, nodes)
                    while randNode in linksTemp[j] or randNode == j:
                        randNode = np.random.randint(0, nodes)
                    #randNode = random.choice(list(range(0, nodes)).remove(links[j], j)) <- sneller als dit lukt
                    links = addlink(links, j, randNode)
                else:
                    links = addlink(links, i, j)
    return links

def createnetwork(nodes, k, p):
    return {'nodes': nodes, 'links': createWattsStrogatz(nodes, k, p)}

def roundtoeven(a):
    if type(a) != int:
        raise print('k should be an integer')
    if a%2 == 0:
        return a
    else:
        return a - 1

def addlink(links, u, v):
    links[u].append(v)
    links[v].append(u)
    return links

def removelink(links, u, v):
    links[u].remove(v)
    links[v].remove(u)
    return links

In [ ]:
def start(Network, exposedPerc, local):
    Network['currentState'] = [ST.S]*Network['nodes']
    Network['currentColor'] = [STcolor.S]*Network['nodes']
    Network['socialDistanceTimestep'] = np.nan
    exposedNumber = exposedPerc*Network['nodes']
    if local:
        exposed = startperc(Network, exposedNumber)
    else:
        exposed = []
        i = 0
        while i < exposedNumber:
            randomN = random.randint(0, Network['nodes'] - 1)
            if randomN not in exposed:
                exposed.append(randomN)
                i += 1
    for i in exposed:
            Network['currentState'][i] = ST.E
            Network['currentColor'][i] = STcolor.E
            
    return Network

def startperc(Network, exposedNumber):
    exposed = [random.randint(0, Network['nodes'] - 1)]
    while len(exposed) < exposedNumber:
        exposedNew = exposed.copy()
        for i in exposedNew:
            for connectedToI in Network['links'][i]:
                if len(exposedNew) < exposedNumber:
                    if connectedToI not in exposedNew:
                        exposedNew.append(connectedToI)
                else:
                    return exposedNew
        if len(exposed) == len(exposedNew):
            return startperc(Network, exposedNumber)
        exposed = exposedNew
    
    return exposed

In [ ]:
# Create list of the next state after 1 timestep
def updateSEIR(Network, beta, delta, gamma):
    Network['newState'] = []
    for i in range(Network['nodes']):
        if Network['currentState'][i] == ST.S:
            Phealthy = 1
            for j in Network['links'][i]:
                if Network['currentState'][j] == ST.I:
                    Phealthy *= (1 - beta)
            PSick = 1 - Phealthy
            randomN = random.uniform(0, 1)
            if randomN < PSick:
                Network['newState'].append(ST.E)
                Network['currentColor'][i] = STcolor.E
            else:
                Network['newState'].append(Network['currentState'][i])
        elif Network['currentState'][i] == ST.E:
            randomN = random.uniform(0, 1)
            if randomN < delta:
                Network['newState'].append(ST.I)
                Network['currentColor'][i] = STcolor.I
            else:
                Network['newState'].append(Network['currentState'][i])
        elif Network['currentState'][i] == ST.I:
            randomN = random.uniform(0, 1)
            if randomN < gamma:
                Network['newState'].append(ST.R)
                Network['currentColor'][i] = STcolor.R
            else:
                Network['newState'].append(Network['currentState'][i])
        else:
            Network['newState'].append(Network['currentState'][i])
    Network['currentState'] = Network['newState']
    
    return Network

# Creates list holding all information of the situation in every time step 
# and plots color-coded network of every timestep if plotNetwork is True
def spreaddiseaseSEIR(Network, timesteps, beta, delta, gamma, socialDistance = False, socialDistancePerc = 0, removePerc = 0, split = False):
    Network['dataStates'] = [Network['currentState']]
    Network['dataColors'] = [Network['currentColor'].copy()]
    Network = countingSEIR(Network)
    
    for i in range(timesteps):
        if socialDistance:
            if Network['I'][-1] >= Network['nodes']*socialDistancePerc:
                if split:
                    originalNetwork = copy.deepcopy(Network)
                Network['links'] = removeedges(Network['links'], removePerc)
                Network['socialDistanceTimestep'] = i
                socialDistance = False
        
        Network = updateSEIR(Network, beta, delta, gamma)
        Network['dataStates'].append(Network['currentState'])
        Network['dataColors'].append(Network['currentColor'].copy())
        Network = countingSEIR(Network)
    
    if Network['socialDistanceTimestep'] != np.nan and split:
        for i in range(Network['socialDistanceTimestep'], timesteps):
            originalNetwork = updateSEIR(originalNetwork, beta, delta, gamma)
            originalNetwork['dataStates'].append(originalNetwork['currentState'])
            originalNetwork['dataColors'].append(originalNetwork['currentColor'].copy())
            originalNetwork = countingSEIR(originalNetwork)
        
    if split:
        return Network, originalNetwork
    else:
        return Network

# Count the amount of people in each state for every timestep
def countingSEIR(Network):
    counting = np.bincount(Network['currentState'], minlength = 4)
    Network['S'].append(counting[ST.S])
    Network['E'].append(counting[ST.E])
    Network['I'].append(counting[ST.I])
    Network['R'].append(counting[ST.R])
        
    return Network

# Executes all the functions above 
# and plots the evolution in all states with respect to time if plotStates == True
def everythingSEIR(Network, timesteps, beta, delta, gamma, plotStates = False, socialDistance = False, socialDistancePerc = 0, removePerc = 0, split= False):
    Network['time'] = np.arange(0, timesteps + 1, 1)
    Network['S'] = []
    Network['E'] = []
    Network['I'] = []
    Network['R'] = []
    
    if split:
        Network, originalNetwork = spreaddiseaseSEIR(Network, timesteps, beta, delta, gamma, socialDistance, socialDistancePerc, removePerc, split)
    else:
        Network = spreaddiseaseSEIR(Network, timesteps, beta, delta, gamma, socialDistance, socialDistancePerc, removePerc)
        
    if plotStates:
        if split:
            figData, (axData1, axData2) = plt.subplots(nrows= 1, ncols= 2, figsize = [2*PL.zoom, PL.zoom])
            
            axData1.plot(originalNetwork['time'], originalNetwork['S'], label = 'S', color= STcolor.S)
            axData1.plot(originalNetwork['time'], originalNetwork['E'], label = 'E', color= STcolor.E)
            axData1.plot(originalNetwork['time'], originalNetwork['I'], label = 'I', color= STcolor.I)
            axData1.plot(originalNetwork['time'], originalNetwork['R'], label = 'R', color= STcolor.R)
            
            axData1.legend(fontsize= 12)
            axData1.set_ylim(0, nodes)
            axData1.set_xlim(0, timesteps)
            axData1.tick_params(axis= 'both', which= 'major', labelsize= 14)
            axData1.grid()

            axData2.plot(Network['time'], Network['S'], label = 'S', color= STcolor.S)
            axData2.plot(Network['time'], Network['E'], label = 'E', color= STcolor.E)
            axData2.plot(Network['time'], Network['I'], label = 'I', color= STcolor.I)
            axData2.plot(Network['time'], Network['R'], label = 'R', color= STcolor.R)
            axData2.axvline(Network['socialDistanceTimestep'])

            axData2.legend(fontsize= 12)
            axData2.set_ylim(0, nodes)
            axData2.set_xlim(0, timesteps)
            axData2.tick_params(axis= 'both', which= 'major', labelsize= 14)
            axData2.grid()
            plt.show()
            
            
        
        else:
            figData = plt.figure()

            plt.plot(Network['time'], Network['S'], label = 'S', color= STcolor.S)
            plt.plot(Network['time'], Network['E'], label = 'E', color= STcolor.E)
            plt.plot(Network['time'], Network['I'], label = 'I', color= STcolor.I)
            plt.plot(Network['time'], Network['R'], label = 'R', color= STcolor.R)
            plt.axvline(Network['socialDistanceTimestep'])

            plt.title('', size= 20)
            plt.ylabel('', size= 20)
            plt.xlabel('', size= 20)
            plt.legend(fontsize= 12)
            plt.ylim(0, nodes)
            plt.xlim(0, timesteps)
            plt.tick_params(axis= 'both', which= 'major', labelsize= 14)
            plt.grid()

            plt.show()
        
        
    if split:
        return Network, originalNetwork
    else:
        return Network
    
def averageSEIR(nodes, k, p, exposedPerc, timesteps, local, beta, delta, gamma, N, plotAverage, averageOverNetwork = 1, socialDistance= False, socialDistancePerc = 0, removePerc = 0, split= False):
    if split:
        averageS = np.zeros(timesteps + 1)
        averageE = np.zeros(timesteps + 1)
        averageI = np.zeros(timesteps + 1)
        averageR = np.zeros(timesteps + 1)
        averageOriginalS = np.zeros(timesteps + 1)
        averageOriginalE = np.zeros(timesteps + 1)
        averageOriginalI = np.zeros(timesteps + 1)
        averageOriginalR = np.zeros(timesteps + 1)
        averageSocialDistanceTimestep = 0
        for i in range(N):
            Network = createnetwork(nodes, k, p)
            NetworkNew = copy.deepcopy(Network)
            for j in range(averageOverNetwork):
                Network = copy.deepcopy(NetworkNew)
                Network = start(Network, exposedPerc, local)
                Network, originalNetwork = everythingSEIR(Network, timesteps, beta, delta, gamma, plotStates= False, socialDistance= socialDistance, socialDistancePerc= socialDistancePerc, removePerc= removePerc, split= split)
                averageS += Network['S']
                averageE += Network['E']
                averageI += Network['I']
                averageR += Network['R']
                averageOriginalS += originalNetwork['S']
                averageOriginalE += originalNetwork['E']
                averageOriginalI += originalNetwork['I']
                averageOriginalR += originalNetwork['R']
                averageSocialDistanceTimestep += Network['socialDistanceTimestep']

        averageS = averageS/(N*averageOverNetwork)
        averageE = averageE/(N*averageOverNetwork)
        averageI = averageI/(N*averageOverNetwork)
        averageR = averageR/(N*averageOverNetwork)
        averageOriginalS = averageOriginalS/(N*averageOverNetwork)
        averageOriginalE = averageOriginalE/(N*averageOverNetwork)
        averageOriginalI = averageOriginalI/(N*averageOverNetwork)
        averageOriginalR = averageOriginalR/(N*averageOverNetwork)
        averageSocialDistanceTimestep = averageSocialDistanceTimestep/(N*averageOverNetwork)
        
        if plotAverage:
            figData, (axData1, axData2) = plt.subplots(nrows= 1, ncols= 2, figsize = [2*PL.zoom, PL.zoom])
            
            axData1.plot(originalNetwork['time'], averageOriginalS, label = 'S', color= STcolor.S)
            axData1.plot(originalNetwork['time'], averageOriginalE, label = 'E', color= STcolor.E)
            axData1.plot(originalNetwork['time'], averageOriginalI, label = 'I', color= STcolor.I)
            axData1.plot(originalNetwork['time'], averageOriginalR, label = 'R', color= STcolor.R)
            
            axData1.legend(fontsize= 12)
            axData1.set_ylim(0, nodes)
            axData1.set_xlim(0, timesteps)
            axData1.tick_params(axis= 'both', which= 'major', labelsize= 14)
            axData1.grid()

            axData2.plot(originalNetwork['time'], averageS, label = 'S', color= STcolor.S)
            axData2.plot(originalNetwork['time'], averageE, label = 'E', color= STcolor.E)
            axData2.plot(originalNetwork['time'], averageI, label = 'I', color= STcolor.I)
            axData2.plot(originalNetwork['time'], averageR, label = 'R', color= STcolor.R)
            axData2.axvline(averageSocialDistanceTimestep)
            
            axData2.legend(fontsize= 12)
            axData2.set_ylim(0, nodes)
            axData2.set_xlim(0, timesteps)
            axData2.tick_params(axis= 'both', which= 'major', labelsize= 14)
            axData2.grid()
            
            plt.show()
            
        return averageS, averageE, averageI, averageR, averageOriginalS, averageOriginalE, averageOriginalI, averageOriginalR, averageSocialDistanceTimestep
    
    else:
        averageS = np.zeros(timesteps + 1)
        averageE = np.zeros(timesteps + 1)
        averageI = np.zeros(timesteps + 1)
        averageR = np.zeros(timesteps + 1)
        
        for i in range(N):
            Network = createnetwork(nodes, k, p)
            NetworkNew = copy.deepcopy(Network)
            for j in range(averageOverNetwork):
                Network = copy.deepcopy(NetworkNew)
                Network = start(Network, exposedPerc, local)
                Network = everythingSEIR(Network, timesteps, beta, delta, gamma, plotStates= False, socialDistance= socialDistance, socialDistancePerc= socialDistancePerc, removePerc= removePerc, split= split)
                averageS += Network['S']
                averageE += Network['E']
                averageI += Network['I']
                averageR += Network['R']
            
        averageS = averageS/(N*averageOverNetwork)
        averageE = averageE/(N*averageOverNetwork)
        averageI = averageI/(N*averageOverNetwork)
        averageR = averageR/(N*averageOverNetwork)
        
        if plotAverage:
            
            figAverage = plt.figure()

            plt.plot(Network['time'], averageS, label = 'S', color= STcolor.S)
            plt.plot(Network['time'], averageE, label = 'E', color= STcolor.E)
            plt.plot(Network['time'], averageI, label = 'I', color= STcolor.I)
            plt.plot(Network['time'], averageR, label = 'R', color= STcolor.R)

            plt.title('', size= 20)
            plt.ylabel('', size= 20)
            plt.xlabel('', size= 20)
            plt.legend(fontsize= 12)
            plt.ylim(0, nodes)
            plt.xlim(0, timesteps)
            plt.tick_params(axis= 'both', which= 'major', labelsize= 14)
            plt.grid()

            plt.show()
    
        return averageS, averageE, averageI, averageR

In [ ]:
def removeedges(links, removePerc):
    if removePerc <= 0.5:
        removeN = round(sum(map(len, links))*removePerc/2)
        for i in range(removeN):
            randomNode = random.choice(random.choices(links, weights= map(len, links), k= 1)[0])
            randomConnectedNode = random.choice(links[randomNode])
            links = removelink(links, randomNode, randomConnectedNode)
        return links
    else:
        addN = round(sum(map(len, links))*(1 - removePerc)/2)
        linksTemp = [ [] for i in range(len(links)) ]
        for i in range(addN):
            randomNode = random.choice(random.choices(links, weights= map(len, links), k= 1)[0])
            randomConnectedNode = random.choice(links[randomNode])
            links = removelink(links, randomNode, randomConnectedNode)
            linksTemp = addlink(linksTemp, randomNode, randomConnectedNode)
        return linksTemp

In [ ]:
nodes = 200
k = 10
p = 0.2
exposedPerc = 0.1
local = False
timesteps = 50
beta = 1/3
delta = 1/8
gamma = 0.05
plotStates = True
Network = createnetwork(nodes, k, p)
Network = start(Network, exposedPerc, local)
Network = everythingSEIR(Network, timesteps, beta, delta, gamma, plotStates= True)

interval = 500
fig, ax = plt.subplots(figsize = [PL.zoom, PL.zoom])
def animate(i):
    if i == 0:
        ax.cla()
        ax.axis('off')
        nx.draw_networkx_edges(Network['links'], Network['pos'], alpha= PL.edgeAlpha, edge_color= PL.edgeColor, ax= ax)
    nx.draw_networkx_nodes(Network['links'], Network['pos'], node_size= PL.nodeSize, node_color= Network['dataColors'][i], ax= ax)
    ax.set_title(i)
aniSEIR = animation.FuncAnimation(fig, animate, frames= timesteps + 1, repeat= True, interval= interval)
plt.show()

In [ ]:
nodes = 500
k = 10
p = 0.3
exposedPerc = 0.1
local = True
days = 50
stepsPerDay = 4
timesteps = days*stepsPerDay
beta = (1/6)/stepsPerDay
delta = (1/5.2)/stepsPerDay
gamma = (1/18)/stepsPerDay

averageOverNetwork = 5
N = 100
plotAverage = True
averageS, averageE, averageI, averageR = averageSEIR(nodes, k, p, exposedPerc, timesteps, local, beta, delta, gamma, N, plotAverage, averageOverNetwork= averageOverNetwork)

In [ ]:
nodes = 300
k = 10
p = 0.2
exposedPerc = 0.1
local = True
stepsPerDay = 4
timesteps = 80*stepsPerDay
beta = (1/6)/stepsPerDay
delta = (1/5.2)/stepsPerDay
gamma = (1/18)/stepsPerDay
plotStates = True

socialDistance = True
removePerc = 0.5
socialDistancePerc = 0.3
split= True

Network = createnetwork(nodes, k, p)
Network = start(Network, exposedPerc, local)
Network, originalNetwork = everythingSEIR(Network, timesteps, beta, delta, gamma, plotStates= plotStates, socialDistance= socialDistance, removePerc= removePerc, socialDistancePerc= socialDistancePerc, split= split)

interval = 500
fig, (ax1, ax2) = plt.subplots(nrows= 1, ncols= 2, figsize = [2*PL.zoom, PL.zoom])
def animate(i):
    if i == 0:
        ax1.cla()
        ax1.axis('off')
        nx.draw_networkx_edges(originalNetwork['links'], Network['pos'], alpha= PL.edgeAlpha, edge_color= PL.edgeColor, ax= ax1)
        
        ax2.cla()
        ax2.axis('off')
        nx.draw_networkx_edges(originalNetwork['links'], Network['pos'], alpha= PL.edgeAlpha, edge_color= PL.edgeColor, ax= ax2)
    
    if i == Network['socialDistanceTimestep']:
        ax2.cla()
        ax2.axis('off')
        nx.draw_networkx_edges(Network['links'], Network['pos'], alpha= PL.edgeAlpha, edge_color= PL.edgeColor, ax= ax2)
    nx.draw_networkx_nodes(originalNetwork['links'], Network['pos'], node_size= PL.nodeSize, node_color= originalNetwork['dataColors'][i], ax= ax1)
    nx.draw_networkx_nodes(Network['links'], Network['pos'], node_size= PL.nodeSize, node_color= Network['dataColors'][i], ax= ax2)
    ax1.set_title(i)
    ax2.set_title(i)
aniSEIR = animation.FuncAnimation(fig, animate, frames= timesteps + 1, repeat= True, interval= interval)
plt.show()
print(Network['socialDistanceTimestep'])

In [ ]:
nodes = 200
k = 10
p = 0.2
exposedPerc = 0.1
local = True
timesteps = 100
beta = 0.2
delta = 1/8
gamma = 0.05
plotAverage = True
N = 100
socialDistance = True
removePerc = 0.5
socialDistancePerc = 0.2
split= True

averageS, averageE, averageI, averageR, averageOriginalS, averageOriginalE, averageOriginalI, averageOriginalR, averageSocialDistanceTimestep = averageSEIR(nodes, k, p, exposedPerc, timesteps, local, beta, delta, gamma, N, plotAverage, socialDistance= socialDistance, socialDistancePerc = socialDistancePerc, removePerc = removePerc, split= split)

In [ ]:
nodes = 500
k = 10
p = 0.3
exposedPerc = 0.1
local = True
days = 50
stepsPerDay = 4
timesteps = days*stepsPerDay
beta = (1/6)/stepsPerDay
delta = (1/5.2)/stepsPerDay
gamma = (1/18)/stepsPerDay

time = np.arange(0, timesteps + 1, 1)
initialCondition = [nodes*(1 - exposedPerc), nodes*exposedPerc, 0, 0]

averageOverNetwork = 5
N = 100
socialDistance = True
socialDistancePerc = 0
split= False
plotAverage = False

averageListS = []
averageListE = []
averageListI = []
averageListR = []
removePercList = list(map(float, np.arange(0, 1, 0.05)))
for removePerc in removePercList:
    averageS, averageE, averageI, averageR = averageSEIR(nodes, k, p, exposedPerc, timesteps, local, beta, delta, gamma, N, plotAverage, averageOverNetwork = averageOverNetwork, socialDistance= socialDistance, socialDistancePerc= socialDistancePerc, removePerc= removePerc, split= split)
    averageListS.append(averageS)
    averageListE.append(averageE)
    averageListI.append(averageI)
    averageListR.append(averageR)
    print(removePerc)

In [ ]:
for i in range(len(removePercList)):
    averageDictionary = {'S': averageListS[i], 'E': averageListE[i], 'I': averageListI[i], 'R': averageListR[i]}
    pandasdf = pd.DataFrame(averageDictionary)
    pandasdf.to_csv(r'dropbox\averageStates' + str(i) + '.csv')

In [ ]:
nodes = 500
k = 10
p = 0.3
exposedPerc = 0.1
local = True
days = 50
stepsPerDay = 4
timesteps = days*stepsPerDay
beta = (1/6)/stepsPerDay
delta = (1/5.2)/stepsPerDay
gamma = (1/18)/stepsPerDay

time = np.arange(0, timesteps + 1, 1)
initialCondition = [nodes*(1 - exposedPerc), nodes*exposedPerc, 0, 0]

averageOverNetwork = 5
N = 100
socialDistance = True
socialDistancePerc = 0
split= False
plotAverage = False

In [ ]:
averageListS = []
averageListE = []
averageListI = []
averageListR = []
removePercList = list(map(float, np.arange(0, 1, 0.05)))
for i in range(len(removePercList)):
    csvRead = pd.read_csv(r'C:\Users\brams\Documents\School\Semester6\Bachelorproef\data\20minlanggerunnedliiiit/averageStates' + str(i) + '.csv')
    averageListS.append(csvRead['S'])
    averageListE.append(csvRead['E'])
    averageListI.append(csvRead['I'])
    averageListR.append(csvRead['R'])

In [ ]:
def linear(x, m, q):
    return m*x + q

def ODEsolveI(time, betaf, deltaf, gammaf):
    def ODE(time, y):
        Spos, Epos, Ipos, Rpos = y
        return [-betaf*Ipos*Spos/nodes, betaf*Ipos*Spos/nodes - deltaf*Epos, deltaf*Epos - gammaf*Ipos, gammaf*Ipos]
    return solve_ivp(ODE, (time[0], time[-1]), initialCondition, t_eval = time)['y'][ST.I]

In [ ]:
fig = plt.figure()
colorlistI = ['darkred', 'red', 'chocolate', 'orange', 'goldenrod', 'darkkhaki', 'yellow', 'yellowgreen', 'limegreen', 'seagreen', 'springgreen', 'turquoise', 'aqua', 'deepskyblue', 'dodgerblue', 'royalblue', 'navy', 'blueviolet', 'purple', 'deeppink']
for i in range(len(removePercList)):
    plt.plot(time, averageListI[i], label= str(int(removePercList[i]*100))+ '%', color= colorlistI[i])
plt.title('', size= 20)
plt.ylabel('', size= 20)
plt.xlabel('', size= 20)
plt.legend(fontsize= 8, loc = 'upper left')
plt.tick_params(axis= 'both', which= 'major', labelsize= 14)
plt.ylim(0, nodes)
plt.grid()

plt.show()

In [ ]:
parIList = []
for i in range(len(removePercList)):
    parI, parICov = optimize.curve_fit(ODEsolveI, time, averageListI[i], p0= [beta, delta, gamma], bounds = [0, 500])
    parIList.append(parI)
    print(i)
parIList

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = len(removePercList), figsize = [PL.zoom*len(removePercList), PL.zoom])
for i in range(len(removePercList)):
    ax[i].plot(time, ODEsolveI(time, parIList[i][0], parIList[i][1], parIList[i][2]))
    ax[i].plot(time, averageListI[i])
    ax[i].set_ylim(0, nodes)
plt.show()

In [ ]:
R0IList = []
for i in range(len(removePercList)):
    R0IList.append(parIList[i][0]/parIList[i][2])
parIR0, parIR0Cov = optimize.curve_fit(linear, removePercList, R0IList)

fig = plt.figure()
removePercPlot = np.arange(0, 1+0.1, 0.1)
plt.plot(removePercList, R0IList, '.')
plt.plot(removePercPlot, linear(np.array(removePercPlot), parIR0[0], parIR0[1]))
plt.show()
print(parIR0[0], parIR0[1])